# Modeling Football Player-Team Relationships Using Neo4j and Python


Under the supervision of: Prof. Manar El-Shazly ❤️

In [2]:
# Cell 1 — Install dependencies (run once)
!pip install neo4j


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Cell 2 — Imports & Neo4j connection
from neo4j import GraphDatabase
import tkinter as tk
from tkinter import ttk, messagebox

# Neo4j credentials
URI      = "neo4j+s://cc7e7431.databases.neo4j.io"
USERNAME = "neo4j"
PASSWORD = "6VhDjKoz1Gkh4qT4DMJAy_qQ6wsI2bRajAJUwOTFkAY"

driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [4]:
# Cell 3 — FULL seed of your football graph (no truncation)

def clear_database(tx):
    tx.run("MATCH (n) DETACH DELETE n")

def create_player(tx, name, age, position, nationality):
    tx.run(
        "MERGE (:Player {name:$name, age:$age, position:$position, nationality:$nationality})",
        name=name, age=int(age), position=position, nationality=nationality
    )

def create_team(tx, name, country, league):
    tx.run(
        "MERGE (:Team {name:$name, country:$country, league:$league})",
        name=name, country=country, league=league
    )

def create_country(tx, name, confederation):
    tx.run(
        "MERGE (:Country {name:$name, confederation:$confederation})",
        name=name, confederation=confederation
    )

def create_stadium(tx, name, city, capacity):
    tx.run(
        "MERGE (:Stadium {name:$name, city:$city, capacity:$capacity})",
        name=name, city=city, capacity=int(capacity)
    )

def create_match(tx, match_id, date, competition):
    tx.run(
        "MERGE (:Match {id:$match_id, date:$date, competition:$competition})",
        match_id=match_id, date=date, competition=competition
    )

def create_coach(tx, name, nationality):
    tx.run(
        "MERGE (:Coach {name:$name, nationality:$nationality})",
        name=name, nationality=nationality
    )

def create_trophy(tx, name, year):
    tx.run(
        "MERGE (:Trophy {name:$name, year:$year})",
        name=name, year=int(year)
    )

def create_plays_for(tx, player, team):
    tx.run(
        "MATCH (p:Player {name:$player}), (t:Team {name:$team}) "
        "MERGE (p)-[:PLAYS_FOR]->(t)",
        player=player, team=team
    )

def create_represents(tx, player, country):
    tx.run(
        "MATCH (p:Player {name:$player}), (c:Country {name:$country}) "
        "MERGE (p)-[:REPRESENTS]->(c)",
        player=player, country=country
    )

def create_plays_international(tx, player, country):
    tx.run(
        "MATCH (p:Player {name:$player}), (c:Country {name:$country}) "
        "MERGE (p)-[:PLAYS_INTERNATIONAL]->(c)",
        player=player, country=country
    )

def create_home_stadium(tx, team, stadium):
    tx.run(
        "MATCH (t:Team {name:$team}), (s:Stadium {name:$stadium}) "
        "MERGE (t)-[:HAS_HOME]->(s)",
        team=team, stadium=stadium
    )

def create_manages(tx, coach, team):
    tx.run(
        "MATCH (c:Coach {name:$coach}), (t:Team {name:$team}) "
        "MERGE (c)-[:MANAGES]->(t)",
        coach=coach, team=team
    )

def create_hosted_at(tx, match_id, stadium):
    tx.run(
        "MATCH (m:Match {id:$match_id}), (s:Stadium {name:$stadium}) "
        "MERGE (m)-[:HOSTED_AT]->(s)",
        match_id=match_id, stadium=stadium
    )

def create_played_in(tx, player, match_id):
    tx.run(
        "MATCH (p:Player {name:$player}), (m:Match {id:$match_id}) "
        "MERGE (p)-[:PLAYED_IN]->(m)",
        player=player, match_id=match_id
    )

def create_won(tx, team, trophy, year):
    tx.run(
        "MATCH (t:Team {name:$team}), (tr:Trophy {name:$trophy, year:$year}) "
        "MERGE (t)-[:WON]->(tr)",
        team=team, trophy=trophy, year=int(year)
    )

def create_held_in(tx, trophy, year, country):
    tx.run(
        "MATCH (tr:Trophy {name:$trophy, year:$year}), (c:Country {name:$country}) "
        "MERGE (tr)-[:HELD_IN]->(c)",
        trophy=trophy, year=int(year), country=country
    )

# — now seed everything exactly as in your original script —
with driver.session() as session:
    session.write_transaction(clear_database)

    # Countries
    countries = [
        ("Argentina", "CONMEBOL"), ("Portugal", "UEFA"),   ("France", "UEFA"),
        ("Egypt",     "CAF"),      ("Spain",    "UEFA"),   ("Brazil", "CONMEBOL"),
        ("Germany",   "UEFA"),     ("England",  "UEFA"),   ("Saudi Arabia", "AFC"),
        ("Croatia",   "UEFA")
    ]
    for c, conf in countries:
        session.write_transaction(create_country, c, conf)

    # Teams
    teams = [
        ("FC Barcelona","Spain","La Liga"),("Real Madrid","Spain","La Liga"),
        ("Liverpool","England","Premier League"),("Paris Saint‑Germain","France","Ligue 1"),
        ("Manchester City","England","Premier League"),("Al‑Nassr","Saudi Arabia","Saudi Pro League"),
        ("Bayern Munich","Germany","Bundesliga"),("Juventus","Italy","Serie A"),
        ("Inter Milan","Italy","Serie A"),("Chelsea","England","Premier League")
    ]
    for name, country, league in teams:
        session.write_transaction(create_team, name, country, league)

    # Players (all 20)
    players = [
        ("Lionel Messi",36,"Forward","Argentina"),("Cristiano Ronaldo",38,"Forward","Portugal"),
        ("Kylian Mbappé",25,"Forward","France"),("Mohamed Salah",32,"Forward","Egypt"),
        ("Sergio Busquets",35,"Midfielder","Spain"),("Neymar Jr.",32,"Forward","Brazil"),
        ("Luka Modrić",38,"Midfielder","Croatia"),("Joshua Kimmich",29,"Midfielder","Germany"),
        ("Trent Alexander‑Arnold",25,"Defender","England"),("Riyad Mahrez",33,"Winger","Algeria"),
        ("Virgil van Dijk",32,"Defender","Netherlands"),("Robert Lewandowski",34,"Forward","Poland"),
        ("Kevin De Bruyne",32,"Midfielder","Belgium"),("Thibaut Courtois",31,"Goalkeeper","Belgium"),
        ("Karim Benzema",35,"Forward","France"),("Sadio Mane",31,"Forward","Senegal"),
        ("Erling Haaland",23,"Forward","Norway"),("Paul Pogba",30,"Midfielder","France"),
        ("Gerard Piqué",36,"Defender","Spain"),("Zlatan Ibrahimović",43,"Forward","Sweden")
    ]
    for n, a, pos, nat in players:
        session.write_transaction(create_player, n, a, pos, nat)
        session.write_transaction(create_represents, n, nat)
        session.write_transaction(create_plays_international, n, nat)

    # Coaches (5)
    coaches = [
        ("Pep Guardiola","Spain"),("Jurgen Klopp","Germany"),
        ("Didier Deschamps","France"),("Hervé Renard","France"),
        ("Antonio Conte","Italy")
    ]
    for n, nat in coaches:
        session.write_transaction(create_coach, n, nat)

    # Stadiums (9)
    stadiums = [
        ("Camp Nou","Barcelona",99354),("Lusail Stadium","Lusail",88000),
        ("Old Trafford","Manchester",74879),("Parc des Princes","Paris",47929),
        ("Santiago Bernabéu","Madrid",81000),("Anfield","Liverpool",54074),
        ("Stamford Bridge","London",40793),("Allianz Arena","Munich",75000),
        ("San Siro","Milan",80018)
    ]
    for n, city, cap in stadiums:
        session.write_transaction(create_stadium, n, city, cap)

    # Home stadiums
    session.write_transaction(create_home_stadium, "FC Barcelona",    "Camp Nou")
    session.write_transaction(create_home_stadium, "Paris Saint‑Germain", "Parc des Princes")
    session.write_transaction(create_home_stadium, "Real Madrid",      "Santiago Bernabéu")
    session.write_transaction(create_home_stadium, "Liverpool",        "Anfield")

    # Manages
    session.write_transaction(create_manages, "Pep Guardiola","Manchester City")
    session.write_transaction(create_manages, "Jurgen Klopp",  "Liverpool")
    session.write_transaction(create_manages, "Didier Deschamps","France")

    # Matches (4)
    matches = [
        ("M001","2022-12-18","FIFA World Cup Final"),
        ("M002","2021-05-29","UEFA Champions League Final"),
        ("M003","2022-11-21","World Cup Group Stage"),
        ("M004","2021-04-17","FA Cup Final")
    ]
    for mid, d, comp in matches:
        session.write_transaction(create_match, mid, d, comp)

    # Hosted at
    session.write_transaction(create_hosted_at, "M001","Lusail Stadium")
    session.write_transaction(create_hosted_at, "M002","Stamford Bridge")
    session.write_transaction(create_hosted_at, "M003","Al Bayt Stadium")
    session.write_transaction(create_hosted_at, "M004","Wembley Stadium")

    # Played in
    session.write_transaction(create_played_in, "Lionel Messi","M001")
    session.write_transaction(create_played_in, "Kylian Mbappé","M001")
    session.write_transaction(create_played_in, "Cristiano Ronaldo","M002")
    session.write_transaction(create_played_in, "Mohamed Salah","M003")

    # Trophies (4)
    trophies = [
        ("FIFA World Cup",2022),("UEFA Champions League",2021),
        ("Copa America",2021),("AFCON",2021)
    ]
    for t, y in trophies:
        session.write_transaction(create_trophy, t, y)

    # Won
    session.write_transaction(create_won, "Argentina","FIFA World Cup",2022)
    session.write_transaction(create_won, "Chelsea","UEFA Champions League",2021)
    session.write_transaction(create_won, "Brazil","Copa America",2021)

    # Held in
    session.write_transaction(create_held_in, "FIFA World Cup",2022,"Qatar")
    session.write_transaction(create_held_in, "UEFA Champions League",2021,"England")
    session.write_transaction(create_held_in, "Copa America",2021,"Brazil")

print("✅ Football graph database created successfully!")

C:\Users\ZBook\AppData\Local\Temp\ipykernel_20424\1930401038.py:113: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(clear_database)
C:\Users\ZBook\AppData\Local\Temp\ipykernel_20424\1930401038.py:123: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_country, c, conf)
C:\Users\ZBook\AppData\Local\Temp\ipykernel_20424\1930401038.py:134: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_team, name, country, league)
C:\Users\ZBook\AppData\Local\Temp\ipykernel_20424\1930401038.py:150: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_player, n, a, pos, nat)
C:\Users\ZBook\AppData\Local\Temp\ipykernel_20424\1930401038.py:151: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_represents, n, nat)
C:\Users\ZBook\AppDa

✅ Football graph database created successfully!


In [ ]:
# Cell 4 — CustomTkinter Cypher‑Console GUI (fixed syntax)

import customtkinter as ctk
from tkinter import ttk, messagebox

# apply theme
ctk.set_appearance_mode("dark")      # "System" | "Dark" | "Light"
ctk.set_default_color_theme("blue")  # "blue" | "green" | "dark‑blue"

def run_cypher(query: str):
    with driver.session() as sess:
        return [rec.data() for rec in sess.run(query)]

# build window
app = ctk.CTk()
app.title("Neo4j Cypher Console")
app.geometry("900x600")

container = ctk.CTkFrame(master=app, corner_radius=12)
container.pack(fill="both", expand=True, padx=20, pady=20)

# Query label
label = ctk.CTkLabel(master=container, text="Enter Cypher Query:", anchor="w")
label.grid(row=0, column=0, columnspan=3, sticky="we", padx=10, pady=(5, 0))

# Query textbox
txt_query = ctk.CTkTextbox(master=container, height=120, corner_radius=8)
txt_query.grid(row=1, column=0, columnspan=3, sticky="we", padx=10, pady=10)

# Results frame
result_frame = ctk.CTkFrame(master=container, corner_radius=8)
result_frame.grid(row=3, column=0, columnspan=3, sticky="nsew", padx=10, pady=(0, 10))

def on_run():
    query = txt_query.get("1.0", "end").strip()
    if not query:
        messagebox.showerror("Error", "Please enter a query.")
        return

    try:
        records = run_cypher(query)
    except Exception as e:
        messagebox.showerror("Query Failed", str(e))
        return

    # clear previous results
    for widget in result_frame.winfo_children():
        widget.destroy()

    if not records:
        empty = ctk.CTkLabel(master=result_frame, text="No records returned.")
        empty.pack(pady=20)
        return

    # build Treeview for results
    cols = list(records[0].keys())
    tree = ttk.Treeview(result_frame, columns=cols, show="headings", height=12)
    for col in cols:
        tree.heading(col, text=col)
        tree.column(col, width=120, anchor="w")
    tree.pack(fill="both", expand=True, padx=10, pady=10)

    # insert rows
    for rec in records:
        tree.insert("", "end", values=[rec[col] for col in cols])

# Run button
run_btn = ctk.CTkButton(master=container, text="Run Query", command=on_run, corner_radius=8)
run_btn.grid(row=2, column=0, columnspan=3, pady=(0, 10))

# allow expansion
container.rowconfigure(3, weight=1)
container.columnconfigure((0, 1, 2), weight=1)

app.mainloop()
